## Cvičenie 10 - úvod do Scikit-learn

Cieľom príkladov v tejto lekcii je predstaviť knižnicu Scikit-learn. Scikit-learn je knižnica algoritmov strojového učenia pre jazyk Python a obsahuje implementácie mnohých algoritmov pre klasifikáciu, regresiu, zhlukovanie využívaných v dátovej analytike. 

Nasledujúce príklady demonštrujú základné používanie knižnice vrátanie konverzie dát do formátu vhodného pre trénovanie modelov, základom spôsobe práce pri trénovaní modelov pomocou tejto knižnice. 

Príklady budú demonštrované na štandardnej databáze Iris na príklade tvorby a vyhodnotenia jednoduchého klasifikačného modelu. 

Najprv podobne ako v predošlých lekciách importujeme používané knižnice.

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

Načítame dátovú množinu s ktorou budeme pracovať do dátového rámca Pandas. Pre kontrolu vypíšeme hlavičku importovaného dátového rámca.

In [ ]:
# pomocou funkcie load_dataset() načítame dataset iris z repozitáru štandardných datasetov do dátového rámca iris
iris = sns.load_dataset("iris")

# vypíšeme hlavičku dátového rámca iris
iris.head()

Pre preskúmanie dátovej množiny použijeme knižnicu Seaborn a vykreslíme párový diagram pre jednotlivé atribúty, s farebným rozlíšením podľa hodnoty cieľového atribútu (`species`). 

In [ ]:
# nastavíme predvolený štýl vykresľovania v knižnici Seaborn pomocou funkcie set()
sns.set()

# vykreslíme párový graf pre všetky atribúty s farebným rozlíšením podľa atribútu species
# nepovinný parameter height definuje veľkosť vykresleného grafu oproti štandardne prednastavenej hodnote
g = sns.pairplot(iris, hue='species', height=2);

### Modelovanie pomocou Scikit-learn

Obvykle modelovanie použitím Scikit-Learn API pozostáva z nasledovných krokov:

* Rozdelenie dát do matice príznakov a vektora hodnôt cieľového atribútu
* Rozdelenie dát do trénovacej a testovacej množiny
* Voľba a importovanie tried Scikit-learn modelu, ktorý budeme vytvárať
* Nastavenie parametrov učiaceho algoritmu
* Naučenie modelu na trénovacích dátach pomocou funkcie `fit()`

Nasledovné kroky sú potom závislé na type modelu, ktorý trénujeme. 
* Pre prediktívne modely - použitie modelu na predikciu cieľového atribútu testovacích dát pomocou funkcie `predict()`
* Pre popisné modely - odvodíme vlastnosti modelu použitím funkcií `transform()` alebo `predict()`


#### Rozdelenie dát do matice príznakov a vektora hodnôt cieľového atribútu

Aby sme mohli dáta použiť pre modelovanie pomocou knižnice Scikit-Learn, potrebujeme z dátového rámca pandas extrahovať maticu príznakov a pole hodnôt cieľového atribútu. Oddelíme tak stĺpce, ktoré budú použité na trénovanie modelov od cieľového atribútu. Obrázok nižšie zobrazuje spôsob, ako bude dátový rámec rozdelený. Toto môžeme realizovať pomocou operácií s dátovými rámcami.

![Obrazok](https://jakevdp.github.io/PythonDataScienceHandbook/figures/05.02-samples-features.png)

Konvencia názvov je taká, že maticu príznakov v kóde ukladáme do premennej začínajúcej písmenom `X`. Predpokladá sa, že matica príznakov je dvojrozmerná, s rozmermi `n_samples` X `n_features` (kde `n_samples` je počet príkladov a `n_features` je počet príznakov (atribútov)) a zvyčajne je tvorená NumPy poľom alebo Pandas dátovým rámcom. 


Okrem matice príznakov pracujeme aj s vektorom hodnôt cieľového atribútu. Tento obvykle označujeme začiatočným písmenom `y`. Pole je obvykle jednorozmerné, s dĺžkou `n_samples` (počet príkladov datasetu), ktorá korešponduje s rozmerom matice príznakov a vo všeobecnosti je tvorený NumPy poľom alebo Pandas stĺpcom. Vektor hodnôt cieľových atribútov môže obsahovať numerické hodnoty alebo diskrétne hodnoty reprezentujúce triedy. 




In [ ]:
# Z dátového rámca iris teda pomocou funkcie drop() odstránime stĺpec s cieľovým atribútom a takýto dátový rámec potom uložíme do X_iris

X_iris = iris.drop('species', axis=1)

# Dimenzionality matice príznakov vypíšeme (rozmer 150 x 4)
X_iris.shape

In [ ]:
# Analogicky pre potreby vytvorenia stĺpca vektora hodnôt priradíme do y_iris hodnoty stĺpca "species" z dátového rámca iris

y_iris = iris['species']

# Dimenzionalitu vektora hodnôt cieľového atribútu vypíšeme (rozmer 150)
y_iris.shape

#### Rozdelenie na trénovaciu a testovaciu množinu

V tomto kroku si ukážeme ako dataset rozdeliť na trénovaciu množinu na ktorej natrénujeme klasifikačný model a testovaciu množinu, ktoú použijeme na jeho evaluáciu. Z knižnice `sklearn` použijeme funkciu `train_test_split()`. Táto funkcia slúži na rozdelenie datasetu na trénovaciu a testovaciu množniu a jej parametrami sú matica vstupných dát (v tomto prípade použijeme `X_iris`), vektor s hodnotami cieľového atribútu (`y_iris`). Parameter `random_state` sa používa pre inicializáciu interného generátora náhodných čísel, ktorý sa použije pri rozdeľovaní príkladov do trénovacej a testovacej množiny. Parameter `test_size` (v rozsahu od 0 po 1) špecifikuje pomer veľkosti testovacej množiny k trénovacej (napr. hodnota 0.5 udáva, že pomer testovacej k trénovacej množine bude 50 k 50).

In [ ]:
from sklearn.model_selection import train_test_split # Naimportujeme z knižnice potrebnú funkciu
Xtrain, Xtest, ytrain, ytest = train_test_split(X_iris, y_iris, test_size=0.5, random_state=1) # Použijeme funkciu na rozdelenie matice príznakov a vektora hodnôt na trénovaciu a testovaciu časť

Výsledkom použitia funkcie `train_test_split()` bude:
* `Xtrain` - matica príznakov trénovacej množiny
* `Xtest` - matica príznakov testovacej množiny
* `ytrain` - vektor hodnôt cieľového atribútu trénovacej množiny
* `ytest` - vektor hodnôt cieľového atribútu testovacej množiny

Môžeme vypísať rozmery a hlavičky jednotlivých matíc a vektorov pomocou funkcií `shape` a `head()`

In [ ]:
# YOUR CODE HERE

#### Trénovanie modelu

Na úvod je potrebné naimportovať triedy modelu, ktorý chceme natrénovať. V knižnici Scikit-Learn, každému modelu zodpovedá Python trieda knižnice. V tomto prípade importujeme triedu `KNeighborsClassifier` zodpovedajúce klasifikátoru na báze k najbližších susedov.

Príkazom `model = KNeighborsClassifier()` inicializujeme model a funkcia `fit()` slúži na jeho natrénovanie. Parametrami funkcie `fit()` sú matica trénovacích dokumentov a zodpovedajúci vektor hodnôt cieľového atribútu trénovacích dokumentov (v tomto prípade `Xtrain` a `ytrain` vytvorené v predošlom kroku).

Vytvorený model (objekt `model`) môžeme použiť pre klasifikáciu príkladov v testovacej množine. To realizujeme pomocou aplikácie funkcie `predict()` na vytvorený model. Jej parametrom je matica príznakov (bez vektora hodnôt cieľového atribútu) testovacej množiny (v tomto prípade `Xtest`). Výstupom je vektor predikcií vypočítaných modelom `y_model`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # Importovanie triedy zodpovedajúcej modelu, ktorý budeme trénovať

model = KNeighborsClassifier()       # Natrénovanie modelu kNN  

model.fit(Xtrain, ytrain)                  # Trénovanie modelu na trénovacej množine 
y_model = model.predict(Xtest)             # Použitie modelu pre predpoveď cieľového atribútu matice príznakov testovacích dát

#### Evaluácia modelu

Jednoduchým spôsobom môžeme overiť presnosť naučeného klasifikátora. Použijeme na to vektor hodnôt cieľového atribútu testovacej množiny (`ytest`), ktorý porovnáme s predikovanými hodnotami (vektor `y_model`). Sklearn umožňuje použiť viacero funkcií pre výpočet rôznych metrík kvality klasifikátorov. V príklade nižšie spočítame presnosť (accuracy), teda pomer správne a nesprávne klasifikovaných príkladov. 

Najprv importujeme potrebnú triedu z knižnice sklearn. Potom príkazom `accuracy_score(ytest, y_model)` vypočítame metriku presnosti. Parametre funkcie sú vektor hodnôt cieľového atribútu testovacej množiny a vektor modelom predikovaných hodnôt.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, y_model)

Pre vizualizáciu toho, ako klasifikátor predikoval hodnoty si môžeme zobraziť 'confusion_matrix()'. Tá nám zobrazí, ako si klasifikátor poradil s klasifikáciou do jednotlivých tried. Na diagonále matice vidíme príklady, ktoré boli klasifikované správne, mimo diagonály vidíme chybne klasifikované príklady. Maticu vykreslíme pomocou funkcie `confusion_matrix()`, kde povinné parametre funkcie sú vektor hodnôt cieľového atrubútu a vektor predikcií. 

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ytest, y_model)
print(cm)

Aby sme vykreslenie matice sprehľadnili, môžeme ju doplniť o titulky pre jednotlivé riadky a stĺpce a hodnoty cieľového atribútu tak, že ju transformujeme na dátový rámec.

In [ ]:
print(pd.DataFrame(confusion_matrix(ytest, y_model, labels=['setosa', 'versicolor','virginica']), index=['true:setosa', 'true:versicolor', 'true:virginica'], columns=['pred:setosa', 'pred:versicolor', 'pred:virginica']))


Maticu samozrejme môžeme vizualizovať pomocou knižnice Seaborn alebo iných. 

In [ ]:
g = sns.heatmap(cm, cmap='magma', annot=True)   # vykreslenie pomocou funkcie heatmap() zo seaborn

#### Rozdelenie dát na trénovaciu, validačnú a testovaciu množinu

V predošlom príklade sme rozdelili dataset na trénovaciu a testovaciu množinu. Metodologicky je ale niekedy dataset vhodné rozdeliť na trénovaciu množinu, na ktorej budeme trénovať model, validačnú množinu, na ktorej budeme ladiť nastavenie parametrov modelu a testovaciu množinu, na ktorú budeme používať na evaluáciu. 

Na tomto príklade si ukážeme ako by sme mohli použiť Scikit-learn na rozdelene datasetu takýmto spôsobom. Použijeme teda opäť funkciu `train_test_split()` ako pre rozdelenie na trénovaciu/testovaciu množinu, ale s tým že ju aplikujeme dvakrát. Na prvýraz rozdelíme dataset do dvoch podmnožín vo zvolenom pomere a vytvoríme tak testovaciu množinu a množinu, ktorú ešte raz rozdelíme do dvoch - na trénovaciu a validačnú. 

Príklad nižšie rozdelí dataset do trénovacej/validačnej/testovacej množiny v pomere 60/20/20.

In [ ]:
# Najprv rozdelíme celý dataset do dvoch tak, že vytvoríme "dočasnú" množinu a testovaciu množinu v pomere 80/20
# Potom rovnakým spôsobom rozdelíme "dočasnú" množinu na trénovaciu a testovaciu. Aby sme zachovali pôvodný pomer, tak tentokrát v pomere 75/25

X_temp, Xtest, y_temp, ytest = train_test_split(X_iris, y_iris, test_size=0.2, random_state=1)

Xtrain, Xval, ytrain, yval = train_test_split(X_temp, y_temp, test_size=0.25, random_state=1)

Výsledkom týchto operácií bude trojica matíc príznakov:
* Xtrain (rozmer 90 x 4)
* Xval (rozmer 30 x 4)
* Xtest (rozmer 30 x 4)

A trojica vektorov hodnôt cieľového atribútu:
* ytrain
* yval
* ytest

Ich rozmery a hlavičky môžete skontrolovať nižšie pomocou funkcií `shape` a `head()`.

In [ ]:
# YOUR CODE HERE

Model potom môžeme natrénovať na trénovacej množine, validačnú množinu môžeme použiť na odladenie parametrov modelu a evaluovať môžeme model na testovacej množine. 

Nižšie uvedený kód natrénuje dva modely k-NN na trénovacej množine pre 2 rôzne hodnoty jeho parametra `k`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # Importovanie triedy zodpovedajúcej modelu, ktorý budeme trénovať

model = KNeighborsClassifier(n_neighbors=3)        # Natrénovanie modelu kNN s hodnotou parametra k=3
model.fit(Xtrain, ytrain)                          # Trénovanie modelu na trénovacej množine 

model2 = KNeighborsClassifier(n_neighbors=5)       # Natrénovanie modelu kNN s hodnotou parametra k=5
model2.fit(Xtrain, ytrain)  

y_model = model.predict(Xval)                      # Rôzne modely použijeme na predikciu na validačnej množine
y_model2 = model2.predict(Xval)                   

print(f"Presnosť prvého modelu: {accuracy_score(yval, y_model)}")         # A porovnáme metriky výkonnosti klasifikátorov na validačnej množine
print(f"Presnosť druhého modelu: {accuracy_score(yval, y_model2)}")

Najlepší z modelov na validačnej množine potom použite na evaluáciu na testovacej množine a vypočítajte jeho presnosť.

In [ ]:
# YOUR CODE HERE

#### N-násobná krížová validácia

Scikit-learn knižnica poskytuje aj funkciu pre krížovú validáciu. Z balíčka `model_selection` môžeme použiť funkciu `cross_val_score()`. Funkcia má tri povinné parametre:
* `model` - model, ktorý chceme vyhodnotiť
* `Xtrain` - matica príznakov trénovacej množiny
* `ytrain` - vektor hodnôt cieľového atribútu trénovacej množiny
* `cv` - parameter definujúci násobnosť krížovej validácie

Parametrom `cv` môže byť celé číslo, ktoré udáva stupeň krížovej validácie (napr. `cv`=5 rozdelí trénovaciu množinu na 5 podmnožín, ktoré sú potom použité na trénovanie a testovanie 5 modelov). Okrem konkrétnej špecifikácie hodnoty parametra vieme definovať špecifický spôsob rozdelenia trénovacej množiny. Funkcia `cross_val_score()` vráti ako výsledok pole metrík - v tomto prípade pole presností (accuracy) klasifikátorov. 

In [ ]:
from sklearn.model_selection import cross_val_score       # Importujeme potrebnú funkciu

score = cross_val_score(model, Xtrain, ytrain, cv=5)      # Použijeme krížovú validáciu pre model, rozdelením trénovacej množiny Xtrain/ytrain na 5 častí
print(score)                                              # Vypíšeme pole výsledkov

Ak chceme v krížovej validácii použiť inú metriku pre vyhodnotenie kvality klasifikátora, prípadne chceme zahrnúť viacero metrík pre vyhodnotenie, potrebujeme použiť funkciu `cross_validate()`. Tá má okrem parametrov špecifikujúcich model, trénovaciu množinu a stupňa krížovej validácie aj parameter `scoring`. Ten obsahuje zoznam metrík, ktoré chceme pre modely vypočítať. V nasledujúcom príklade do zoznamu pridáme metriku `accuracy` (na nasledujúcich cvičeniach si vysvetlíme a ukážeme niektoré ďalšie). Funkcia `cross_validate()` okrem zvolených metrík vypočíta aj časy trénovania modelu `fit_time` a čas potrebný na klasifikáciu nového príkladu `score_time`.

Zoznam všetkých metrík, ktoré môžeme použiť pre evaluáciu klasifikačných modelov môžeme násjť v dokumentácii tu: https://scikit-learn.org/stable/modules/model_evaluation.html#multimetric-scoring
Výstupom funkcie je potom pole metrík pre každú etapu krížovej validácie. Jednotlivé metriky si môžeme pozrieť pomocou príkazu `scores.keys()`.

In [ ]:
from sklearn.model_selection import cross_validate          # Importujeme potrebné funkcie pre krížovú validáciu 

scoring = ['accuracy']                                      # Zvolíme metriky, ktoré chceme pre modely vypočítať

scores = cross_validate(model, Xtrain, ytrain, scoring=scoring, cv=5, return_train_score=False) # Realizujeme samotnú krížovú validáciu
                                                            # parameter return_train_score špecifikuje, či v reporte chceme vrátiť aj výsledok evaluácie na trénovacej množine   
sorted(scores.keys())                                       # Zotriedime si výstupne pole metrík podľa kľúča

In [ ]:
print(scores['test_accuracy']) # Vybrané pole metrík vypíšeme 

### Úlohy

Načítajte dataset `Winequality` z adresára `data` pre 10. cvičenie, ktorý popisuje charakteristiky vín. Tie sú popísané 11 atibútmi, ktoré popisujú rôzne (väčšinou chemické) vlastnosti vín. Cieľovým atribútom, ktorý budeme pomocou prediktívnych modelov predpovedať, je kvalita vína. Všetky atribúty v datasete sú numerické, dataset neobsahuje chýbajúce hodnoty. 

![Wines](https://images.all-free-download.com/images/graphicthumb/fine_red_wine_picture_1_167121.jpg)


Popis jednotlivých atribútov:
* `fixed acidity` - Koncentrácia neprchavých kyselín
* `volatile acidity` - Koncentrácia prchavých kyselín
* `citric acid` - množstvo kyseliny citrónovej vo víne 
* `residual sugar` - množstvo cukru, ktoré zostane vo víne po fermentácii 
* `chlorides` - obsah soli vo víne
* `free sulfur dioxide` - množstvo voľného SO<sub>2</sub> 
* `total sulfur dioxide` - celkové množstvo SO<sub>2</sub> (v malých množstvách nedetekovateľný, vo väčších koncentráciách vplyv na chuť vína)
* `denisty` - hustota (pomer hustoty vína k husote vody)
* `pH` - charakterizuje kyslosť/zásaditosť vína v rozsahu od 0 (veľmi kyslé) po 14 (veľmi zásadité); väčšina vín dosahuje hodnoty pH 3-4 
* `sulphates` - množstvo aditív (napr. antioxidantov), ktoré je merateľné pomocou množstva sulfátov
* `alcohol` - množstvo alkoholu vo víne
* `quality` - cieľový atribút, výsledná "známka" charakterizujúca kvalitu vína, nadobúda celé hodnoty 0 - 10 


#### Úloha 10.1

Pripravte tento dataset pre použitie v knižnici Scikit-learn.
* Najprv spojte do jedného dátového rámca oba súbory `winequality_white.csv` a `winequality_red.csv` 
* Integrované dáta rozdeľte do matice príznakov a vektora hodnôt cieľového atribútu.

In [ ]:
# YOUR CODE HERE

#### Úloha 10.2

Rozdeľte pripravený dataset na trénovaciu a testovaciu množinu v pomere 80/20. Vypíšte rozmery pre maticu príznakov a vektor hodnôt cieľového atribútu pre trénovaciu aj testovaciu množinu.

In [ ]:
# YOUR CODE HERE

#### Úloha 10.3

Natrénujte model k-NN na trénovacej množine so zvolenou hodnotou parametra `k`. Použite 10-násobnú krížovú validáciu pre nájdenie najvhodnejšej hodnoty parametra `k`.

In [ ]:
# YOUR CODE HERE

#### Úloha 10.4

Najlepší model otestujte na testovacej množine. Vypíšte presnosť (accuracy) klasifikátora a vykreslite ľubovoľným spôsobom confusion matrix.

In [ ]:
# YOUR CODE HERE